In [ ]:
import time
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

In [ ]:
db = pd.read_csv('base_de_datos.csv', dtype={'Sorteos': "string"}, parse_dates=['Dates'])

In [ ]:
numbers = pd.DataFrame(columns=[number for number in range(51)]).astype("int64")
numbers = numbers.rename(columns={0: 'Sorteos'})
df_clean = {'Sorteos': "string"}
numbers = numbers.astype(df_clean)

In [ ]:
numbers

In [ ]:
list_of_numbers = [] # index of numbers
list_of_stars = [] # index of stars
for i in range(1, 51):
    list_of_numbers.append(i)
print(list_of_numbers)
for i in range(1, 13):
    list_of_stars.append(i)
print(list_of_stars)

In [ ]:
winners = db.iloc[:, 2:7]

In [ ]:
rows = db[db['Nro1'] == 1]
print(rows)

In [ ]:
for number in winners:
    if number == i

In [ ]:
winners = winners[0:1]

In [ ]:
draws = draws.iloc[:, 0]